<a href="https://colab.research.google.com/github/gelya1709/customer_flows/blob/main/Step_2_Calculation_of_customer_flows.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
import nltk, warnings
import itertools
from statistics import mean
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.neighbors import LocalOutlierFactor
from numpy import where
from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
#file which we got at the step 1
p1=pd.read_csv('customers_with_metrics_recency.csv')
p1.head()


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (2,6,10,14,18,22,26,30,34,38,42,46,50,54,58,62,66,70) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Unnamed: 0,client_id,cluster1,Frequency1,Monetary1,Recency1,cluster2,Frequency2,Monetary2,Recency2,...,Monetary22,Recency22,cluster23,Frequency23,Monetary23,Recency23,cluster24,Frequency24,Monetary24,Recency24
0,0,00008fb32c1c30a8ae8356b5591935e0,champions,13.0,13247.06094,3.0,champions,18.0,17303.25848,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,00097e5a83bb53c0d0fc6dcc2b4939ab,sleeping,2.0,1462.56000,2.0,champions,9.0,6492.43856,12.0,...,327.22000,29.0,loyal,5.0,6415.32318,3.0,loyal,7.0,4159.20496,4.0
2,2,000b598bb8fa552507ee02a4d4096682,loyal,4.0,7848.42674,16.0,loyal,4.0,10502.05378,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,sleeping,1.0,879.23948,22.0
3,3,000f0fb0885f4f2a0366cfdab3bce390,sleeping,1.0,1047.36000,15.0,loyal,2.0,5184.68638,2.0,...,1179.86456,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,00125fe3b05ba65e08e6727837059786,sleeping,1.0,121.00000,24.0,NaN,NaN,NaN,NaN,...,NaN,NaN,sleeping,2.0,165.00000,3.0,sleeping,2.0,2111.95176,2.0


In [ ]:
#create lists for using them later
l=[i for i in range(98)]
clusters=l[2:99:4]
freq=l[3:99:4]
mon=l[4:99:4]
rec=l[5:99:4]
rec

In [ ]:
for column in clusters:
  p1[p1.columns[column]]=p1[p1.columns[column]].fillna('churn')

In [ ]:
#use the created lists to fill the dataset with zero values
for column in freq:
  p1[p1.columns[column]]=p1[p1.columns[column]].fillna(0)
for column in mon:
  p1[p1.columns[column]]=p1[p1.columns[column]].fillna(0)
for column in rec:
  p1[p1.columns[column]]=p1[p1.columns[column]].fillna(0)

In [ ]:
p1.head()

,Unnamed: 0,client_id,cluster1,Frequency1,Monetary1,Recency1,cluster2,Frequency2,Monetary2,Recency2,...,Monetary22,Recency22,cluster23,Frequency23,Monetary23,Recency23,cluster24,Frequency24,Monetary24,Recency24
0,0,00008fb32c1c30a8ae8356b5591935e0,champions,13.0,13247.06094,3.0,champions,18.0,17303.25848,1.0,...,0.00000,0.0,churn,0.0,0.00000,0.0,churn,0.0,0.00000,0.0
1,1,00097e5a83bb53c0d0fc6dcc2b4939ab,sleeping,2.0,1462.56000,2.0,champions,9.0,6492.43856,12.0,...,327.22000,29.0,loyal,5.0,6415.32318,3.0,loyal,7.0,4159.20496,4.0
2,2,000b598bb8fa552507ee02a4d4096682,loyal,4.0,7848.42674,16.0,loyal,4.0,10502.05378,5.0,...,0.00000,0.0,churn,0.0,0.00000,0.0,sleeping,1.0,879.23948,22.0
3,3,000f0fb0885f4f2a0366cfdab3bce390,sleeping,1.0,1047.36000,15.0,loyal,2.0,5184.68638,2.0,...,1179.86456,7.0,churn,0.0,0.00000,0.0,churn,0.0,0.00000,0.0
4,4,00125fe3b05ba65e08e6727837059786,sleeping,1.0,121.00000,24.0,churn,0.0,0.00000,0.0,...,0.00000,0.0,sleeping,2.0,165.00000,3.0,sleeping,2.0,2111.95176,2.0


In [ ]:
p1.drop(columns=['Monetary1_x','name1_y','Frequency1_y','name2_y','Frequency2_y','Monetary2_y'], inplace=True)

In [ ]:
#save correct dataset (with zeros)
p1.to_csv('customers_with_metrics_recency_.csv')

In [ ]:
p1 = p1.loc[:, p1.columns != 'Unnamed: 0']

In [ ]:
p1.rename(columns={'name1_x':'name1','Frequency1_x':'Frequency1','Monetary1_y':'Monetary1','name2_x':'name2','Frequency2_x':'Frequency2','Monetary2_x':'Monetary2'},inplace=True)

In [ ]:
p1.head()

,client_id,name1,Frequency1,Monetary1,name2,Frequency2,Monetary2,name3,Frequency3,Monetary3,...,Monetary21,name22,Frequency22,Monetary22,name23,Frequency23,Monetary23,name24,Frequency24,Monetary24
0,00008fb32c1c30a8ae8356b5591935e0,champions,13.0,13247.06094,champions,18.0,17303.25848,champions,14.0,9624.93812,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,00097e5a83bb53c0d0fc6dcc2b4939ab,sleeping,2.0,1462.56000,loyal,9.0,6492.43856,loyal,8.0,3603.05606,...,2759.7452,NaN,NaN,NaN,sleeping,5.0,6415.32318,sleeping,7.0,4159.20496
2,000b598bb8fa552507ee02a4d4096682,loyal,4.0,7848.42674,loyal,4.0,10502.05378,sleeping,2.0,2247.56144,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sleeping,1.0,879.23948
3,000e39c7851a68fbcb8a53fa19125f3c,sleeping,1.0,7413.69412,churn,0.0,0.00000,churn,0.0,0.00000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,000f0fb0885f4f2a0366cfdab3bce390,sleeping,1.0,1047.36000,sleeping,2.0,5184.68638,sleeping,2.0,4612.86520,...,NaN,sleeping,2.0,1179.86456,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df = p1.drop(p1.columns[freq+mon], axis=1) 

In [ ]:
df.head()

,client_id,cluster1,cluster2,cluster3,cluster4,cluster5,cluster6,cluster7,cluster8,cluster9,...,cluster15,cluster16,cluster17,cluster18,cluster19,cluster20,cluster21,cluster22,cluster23,cluster24
0,00008fb32c1c30a8ae8356b5591935e0,champions,champions,champions,churn,churn,churn,sleeping,loyal,churn,...,churn,churn,churn,churn,churn,churn,churn,churn,churn,churn
1,00097e5a83bb53c0d0fc6dcc2b4939ab,sleeping,champions,loyal,loyal,loyal,sleeping,sleeping,loyal,sleeping,...,sleeping,loyal,churn,sleeping,loyal,loyal,loyal,sleeping,loyal,loyal
2,000b598bb8fa552507ee02a4d4096682,loyal,loyal,sleeping,loyal,churn,sleeping,loyal,loyal,loyal,...,loyal,loyal,sleeping,churn,churn,churn,churn,churn,churn,sleeping
3,000f0fb0885f4f2a0366cfdab3bce390,sleeping,loyal,loyal,sleeping,sleeping,sleeping,churn,churn,sleeping,...,churn,churn,loyal,sleeping,churn,sleeping,churn,sleeping,churn,churn
4,00125fe3b05ba65e08e6727837059786,sleeping,churn,churn,churn,sleeping,churn,sleeping,sleeping,sleeping,...,sleeping,churn,sleeping,churn,churn,sleeping,sleeping,churn,sleeping,sleeping


In [ ]:
#change names to the correct ones
df_r=df.replace('churn','4: churn').replace('champions','3: champions').replace('loyal','2: loyal').replace('sleeping','1: sleeping')
df_r.columns=df_r.columns.str.replace('cluster','period')
df_r.head()

,client_id,period1,period2,period3,period4,period5,period6,period7,period8,period9,...,period15,period16,period17,period18,period19,period20,period21,period22,period23,period24
0,00008fb32c1c30a8ae8356b5591935e0,3: champions,3: champions,3: champions,4: churn,4: churn,4: churn,1: sleeping,2: loyal,4: churn,...,4: churn,4: churn,4: churn,4: churn,4: churn,4: churn,4: churn,4: churn,4: churn,4: churn
1,00097e5a83bb53c0d0fc6dcc2b4939ab,1: sleeping,3: champions,2: loyal,2: loyal,2: loyal,1: sleeping,1: sleeping,2: loyal,1: sleeping,...,1: sleeping,2: loyal,4: churn,1: sleeping,2: loyal,2: loyal,2: loyal,1: sleeping,2: loyal,2: loyal
2,000b598bb8fa552507ee02a4d4096682,2: loyal,2: loyal,1: sleeping,2: loyal,4: churn,1: sleeping,2: loyal,2: loyal,2: loyal,...,2: loyal,2: loyal,1: sleeping,4: churn,4: churn,4: churn,4: churn,4: churn,4: churn,1: sleeping
3,000f0fb0885f4f2a0366cfdab3bce390,1: sleeping,2: loyal,2: loyal,1: sleeping,1: sleeping,1: sleeping,4: churn,4: churn,1: sleeping,...,4: churn,4: churn,2: loyal,1: sleeping,4: churn,1: sleeping,4: churn,1: sleeping,4: churn,4: churn
4,00125fe3b05ba65e08e6727837059786,1: sleeping,4: churn,4: churn,4: churn,1: sleeping,4: churn,1: sleeping,1: sleeping,1: sleeping,...,1: sleeping,4: churn,1: sleeping,4: churn,4: churn,1: sleeping,1: sleeping,4: churn,1: sleeping,1: sleeping


In [ ]:
#calculate the number of customers in each cluster in each month
a=df_r[['period1', 'client_id']].groupby('period1').count()
a1=a.reset_index()
a1.columns=['column', 'count1']
for column in df_r.columns.to_list()[2:]:
  aa=df_r[[column, 'client_id']].groupby(column).count()
  aaa=aa.reset_index()
  aaa.columns=['column', column]
  a1=a1.merge(aaa, on='column', how='left')
a1

,column,count1,period2,period3,period4,period5,period6,period7,period8,period9,...,period15,period16,period17,period18,period19,period20,period21,period22,period23,period24
0,1: sleeping,5536,5412,5915,5711,5818,5965,5772,5694,5766,...,5989,3971,4596,5369,5356,5622,5604,5651,5342,5869
1,2: loyal,8447,8664,9350,8787,8784,8525,8523,9091,9206,...,9151,6589,7207,8027,8465,8445,8091,8190,8277,9914
2,3: champions,1211,1099,1113,999,1008,1009,967,1041,1150,...,1226,1068,962,761,1025,990,1036,843,965,1234
3,4: churn,18724,18743,17540,18421,18308,18419,18656,18092,17796,...,17552,22290,21153,19761,19072,18861,19187,19234,19334,16901


In [ ]:
a2=a1.T
a2.head()

In [ ]:
a2=a2.iloc[1:,0:]
a2.columns=['sleeping', 'loyal',	'champions', 'churn']
a2

,sleeping,loyal,champions,churn
count1,5536,8447,1211,18724
period2,5412,8664,1099,18743
period3,5915,9350,1113,17540
period4,5711,8787,999,18421
period5,5818,8784,1008,18308
period6,5965,8525,1009,18419
period7,5772,8523,967,18656
period8,5694,9091,1041,18092
period9,5766,9206,1150,17796
period10,6230,8488,954,18246


In [ ]:
#save this statistics
a2.to_csv('count_of_clusters_24.csv')

In [ ]:
#create dataset to save number of flows
df=pd.DataFrame(columns=['sleeping_to_sleeping','sleeping_to_loyal','sleeping_to_champions','sleeping_to_churn','loyal_to_sleeping','loyal_to_loyal','loyal_to_champions','loyal_to_churn','champions_to_sleeping','champions_to_loyal','champions_to_champions','champions_to_churn','churn_to_sleeping','churn_to_loyal','churn_to_champions','churn_to_churn'])

In [ ]:
#code to calculate the number of customers in each flow
all=[]
for stroka in [0,1,2,3]:
  col=0
  spisok=[]
  for period in range(1,24):

    df2=df_r[['client_id', f'period{str(period)}', f'period{str(period+1)}']]
    output = df2.pivot_table(index=f'period{str(period+1)}', columns=f'period{str(period)}', values='client_id', aggfunc='count', margins=True,margins_name='Grand Total')
 
    #it is necessary to change the numbers in range(...): 0,1 then 1,2 then 2,3 and finally 3,4
    for i in range(3,4):
      #output.iloc[int(i),:]=round((output.iloc[int(i),:]/output.iloc[4,:])*100,1)
      outputt=output.iloc[:4, :4]
      #print(outputt)
      #print(outputt.iloc[stroka,int(i)])
    spisok.append(outputt.iloc[stroka,int(i)])

  #and it is necessary to change columns in dataset: 0:4 then 4:8 then 8:12 then 12:16  
  df[df.iloc[:,12:16].columns[int(stroka)]]=spisok


In [ ]:
#final dataset
df.to_csv('count_of_flows.csv')